In [36]:
import numpy as np
import torch
import tensorflow as tf
import transformers

from decision_transformer import Block

In [6]:
state_dim = 171
act_dim = 80
hidden_size = state_dim + act_dim
inputs = tf.random.uniform(shape=(1,state_dim))

optimizer = tf.optimizers.Adam()


In [ ]:
player_input = [(1,2), (3,4), (5,6)]
embed_player = tf.keras.layers.Embedding(7*7,  hidden_size)
test1 = embed_player(player_input)

test1

In [11]:
embed_state = tf.keras.layers.Embedding(state_dim * act_dim + 1,  hidden_size)
test1 = embed_state(inputs)

#embedding got no trainable weights
with tf.GradientTape() as tape:
    gradients = tape.gradient(test1, embed_state.trainable_weights)
    # optimizer.apply_gradients(zip(gradients, embed_state.trainable_weights))
    print(gradients)

test1

[None]


<tf.Tensor: shape=(1, 171, 251), dtype=float32, numpy=
array([[[-0.02315578, -0.0099403 ,  0.01010556, ...,  0.00455163,
          0.04116095,  0.01053923],
        [-0.02315578, -0.0099403 ,  0.01010556, ...,  0.00455163,
          0.04116095,  0.01053923],
        [-0.02315578, -0.0099403 ,  0.01010556, ...,  0.00455163,
          0.04116095,  0.01053923],
        ...,
        [-0.02315578, -0.0099403 ,  0.01010556, ...,  0.00455163,
          0.04116095,  0.01053923],
        [-0.02315578, -0.0099403 ,  0.01010556, ...,  0.00455163,
          0.04116095,  0.01053923],
        [-0.02315578, -0.0099403 ,  0.01010556, ...,  0.00455163,
          0.04116095,  0.01053923]]], dtype=float32)>

In [44]:
inputs = np.array([
    [[1,1,1,1],[2,2,2,2],[3,3,3,3],[4,4,4,4]],
    [[1,1,1,1],[2,2,2,2],[3,3,3,3],[4,4,4,4]],
    [[1,1,1,1],[2,2,2,2],[3,3,3,3],[4,4,4,4]],
    [[1,1,1,1],[2,2,2,2],[3,3,3,3],[4,4,4,4]],
    [[1,1,1,1],[2,2,2,2],[3,3,3,3],[4,4,4,4]]
    ], dtype=np.float32)
inputs = np.expand_dims(inputs, axis=-1)
embed_state = tf.keras.layers.Conv2D(state_dim, (2,2), input_shape=(4,4,1), dtype=tf.float32)
test1 = embed_state(inputs)
test1.shape

TensorShape([5, 3, 3, 171])

In [35]:
embed_ln = tf.keras.layers.LayerNormalization(axis = -1)
test2 = embed_ln(test1)

with tf.GradientTape() as tape:
    gradients = tape.gradient(test2, embed_ln.trainable_weights)
    # optimizer.apply_gradients(zip(gradients, embed_state.trainable_weights))
    print(gradients)

test2.shape

[None, None]


InvalidArgumentError: Exception encountered when calling layer 'reshape' (type Reshape).

{{function_node __wrapped__Reshape_device_/job:localhost/replica:0/task:0/device:CPU:0}} Input to reshape is a tensor with 1368 values, but the requested shape requires a multiple of 27360 [Op:Reshape]

Call arguments received by layer 'reshape' (type Reshape):
  • inputs=tf.Tensor(shape=(2, 2, 2, 171), dtype=float32)

In [6]:
transformer = Block(hidden_dim = hidden_size, num_heads = 3, key_dim = state_dim)

# print(transformer.trainable_weights)
with tf.GradientTape() as tape:
    test3 = transformer(test2) #this must be within tape so that it will be under the scope
    gradients = tape.gradient(test3, transformer.trainable_variables)
    optimizer.apply_gradients(zip(gradients, embed_state.trainable_variables))
    print("gradients:", gradients)

# test3

gradients: [<tf.Tensor: shape=(251,), dtype=float32, numpy=
array([ 2.51015854e+01,  9.23164845e+00, -2.47162724e+01,  3.89675369e+01,
       -1.97441273e+01, -3.98406677e+01, -6.39576263e+01, -7.99229097e+00,
        5.48156319e+01,  1.78523941e+01, -3.41536951e+00, -1.18047953e+01,
        1.58236313e+01,  6.93269577e+01, -1.59712934e+01,  7.17024460e+01,
        1.57805414e+01, -4.85543823e+01, -2.40139885e+01, -5.66521797e+01,
       -1.05977936e+01,  1.55910826e+01, -9.92734814e+00,  6.12020922e+00,
        2.58887501e+01, -1.18462059e+02, -2.16151447e+01, -1.37414455e+01,
        4.33698702e+00,  2.07197781e+01,  1.73766060e+01,  3.97143587e-02,
        3.00575466e+01, -1.06804438e+01,  2.73406010e+01,  2.02389870e+01,
       -4.48775558e+01, -4.81777039e+01,  2.43571033e+01,  2.40604820e+01,
       -1.38593245e+01,  1.74764080e+01, -1.99776042e+00,  4.34767609e+01,
       -4.46089096e+01,  1.59361219e+01,  7.19152527e+01,  6.78248596e+01,
       -1.50732975e+01,  3.18005729e+00,

In [1]:
test3.shape

NameError: name 'test3' is not defined

In [15]:
predict_action = tf.keras.Sequential(
   [*([tf.keras.layers.Dense(act_dim)] + 
      [tf.keras.layers.Dense(act_dim, activation = 'tanh')] +
      [tf.keras.layers.Flatten()] +
      [tf.keras.layers.Dense(act_dim)]
      )
   ]
)
predict_action.build((1,171,251))

optimizer = tf.optimizers.Adam()

optimizer.build(predict_action.trainable_variables)

with tf.GradientTape() as tape:
    test4 = predict_action(test3) #this must be within tape so that it will be under the scope
    gradients = tape.gradient(test4, predict_action.trainable_variables)
    optimizer.apply_gradients(zip(gradients, predict_action.trainable_variables))
    print("gradients:", gradients)



gradients: [<tf.Tensor: shape=(251, 80), dtype=float32, numpy=
array([[ 2.3629026 , -1.5478299 , -0.299134  , ...,  0.37742007,
         0.08496375,  1.4312353 ],
       [ 2.3337638 , -1.556642  , -0.293323  , ...,  0.31776902,
         0.09651706,  1.4166787 ],
       [ 2.4275131 , -1.6343493 , -0.29290688, ...,  0.32586277,
         0.10107979,  1.4833175 ],
       ...,
       [-1.2917153 ,  0.8071247 ,  0.18497251, ..., -0.11710357,
         0.16742167, -0.9609259 ],
       [ 2.417749  , -1.694185  , -0.30428913, ...,  0.5763366 ,
        -0.06639215,  1.8385606 ],
       [-2.0241842 ,  0.70327306,  0.55080026, ..., -0.06607601,
         0.482064  , -0.66603327]], dtype=float32)>, <tf.Tensor: shape=(80,), dtype=float32, numpy=
array([ 1.2809769 , -0.86032575, -0.15935689,  1.192809  ,  0.52848315,
       -0.21425591, -0.00674458,  0.27568015,  0.8141367 , -0.24001993,
        0.546695  , -0.06959213, -0.3090031 , -0.8599761 ,  0.06410192,
       -1.013059  , -0.21441837, -0.27671108

In [47]:
import sys

for p in sys.path:
    print(p)

c:\Users\P1361755\OneDrive - Singtel\Desktop\learning_code\quoridor_learn_RL\dt_models
c:\Users\P1361755\AppData\Local\Programs\Python\Python310\python310.zip
c:\Users\P1361755\AppData\Local\Programs\Python\Python310\DLLs
c:\Users\P1361755\AppData\Local\Programs\Python\Python310\lib
c:\Users\P1361755\AppData\Local\Programs\Python\Python310

C:\Users\P1361755\AppData\Roaming\Python\Python310\site-packages
C:\Users\P1361755\AppData\Roaming\Python\Python310\site-packages\win32
C:\Users\P1361755\AppData\Roaming\Python\Python310\site-packages\win32\lib
C:\Users\P1361755\AppData\Roaming\Python\Python310\site-packages\Pythonwin
c:\Users\P1361755\AppData\Local\Programs\Python\Python310\lib\site-packages


In [46]:
inputs = np.array([
    [[1,1,1,1],[2,2,2,2],[3,3,3,3],[4,4,4,4]],
    [[1,1,1,1],[2,2,2,2],[3,3,3,3],[4,4,4,4]],
    [[1,1,1,1],[2,2,2,2],[3,3,3,3],[4,4,4,4]],
    [[1,1,1,1],[2,2,2,2],[3,3,3,3],[4,4,4,4]],
    [[1,1,1,1],[2,2,2,2],[3,3,3,3],[4,4,4,4]]
    ], dtype=np.float32)
inputs = np.expand_dims(inputs, axis=-1)

embed_state = tf.keras.layers.Conv2D(state_dim, (2,2), input_shape=(4,4,1), dtype=tf.float32)
# compress_state = tf.keras.layers.Dense(state_dim) #too small to capture
# embed_state_reshape = tf.keras.layers.Reshape((state_dim, 1))

embed_ln = tf.keras.layers.LayerNormalization(axis = -1)
transformer = Block(hidden_dim = state_dim, num_heads = 3, key_dim = state_dim)
predict_action = tf.keras.Sequential(
   [*([tf.keras.layers.Dense(act_dim)] + 
      [tf.keras.layers.Dense(act_dim, activation = 'tanh')] +
      [tf.keras.layers.Reshape((-1, act_dim*3*3))] +
      [tf.keras.layers.Dense(act_dim)]
      )
   ]
)

model_first = tf.keras.Sequential([
        embed_state,
      #   embed_state_reshape,
        embed_ln,
       transformer,
       predict_action 
        ])

model_first.build((None,state_dim))

optimizer = tf.optimizers.Adam()
optimizer.build(model_first.trainable_variables)

outputs = tf.random.uniform(shape=(1,act_dim))

with tf.GradientTape() as tape:
    
    test_final = model_first(inputs) #this must be within tape so that it will be under the scope
    loss = tf.math.reduce_mean((test_final - outputs)**2)
    gradients = tape.gradient(loss, model_first.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model_first.trainable_variables))
    print("gradients:", gradients)

test_final.shape


gradients: [<tf.Tensor: shape=(2, 2, 1, 171), dtype=float32, numpy=
array([[[[ 0.12345045, -0.593756  ,  0.1759058 , -0.20093486,
           0.02769797,  0.12401575,  0.16212435,  0.08847529,
           0.01941985, -0.03952435,  0.5277617 ,  0.04945179,
           0.15276845, -0.31897134, -0.04965761,  0.51908654,
          -0.18413597,  0.4276465 , -0.4991109 , -0.12084794,
          -0.54113716, -0.01375109,  0.19767146, -0.20074   ,
           0.05331306,  0.07048173,  0.11256969, -0.6696418 ,
           0.29668158, -0.2833424 ,  0.07593494,  0.12082781,
           0.01296747,  0.14003365,  0.37505794,  0.14525266,
          -0.10178337,  0.10430254, -0.47456193, -0.65032464,
           0.11890005,  0.14897792,  0.17220463, -0.19955863,
           0.21793987,  0.2310727 ,  0.06962965,  0.21002407,
           0.12341407,  0.19383031,  0.14702556, -0.271384  ,
          -0.01694594,  0.5283311 , -0.07241267, -0.24808832,
          -0.05646674,  0.12021807, -0.12628116,  0.29088366,
  

TensorShape([5, 1, 80])

In [27]:
model_first.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 251)         3433931   
                                                                 
 layer_normalization_1 (Lay  (None, None, 251)         502       
 erNormalization)                                                
                                                                 
 block (Block)               (None, None, 251)         1023611   
                                                                 
 sequential_8 (Sequential)   (None, 80)                1121120   
                                                                 
Total params: 5579164 (21.28 MB)
Trainable params: 5579164 (21.28 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
